In [1]:
import sys
import pyranges as pr
import cerberus
import pandas as pd


In [4]:
def make_hier_entry(df, how='t'):
    """
    kind {'g','t'}
    """
    agg_dict = {'min_coord': 'min', 'max_coord': 'max'}
    t_df = df.copy(deep=True)
    t_df['min_coord'] = t_df[['Start', 'End']].min(axis=1)
    t_df['max_coord'] = t_df[['Start', 'End']].max(axis=1)
    if how == 't':
        gb_cols = ['Chromosome', 'Strand', 'gene_name',
                   'gene_id', 'transcript_id', 'transcript_name',
                   'tss_id', 'tes_id',
                   'new_transcript_id', 'original_transcript_id',
                   'original_transcript_name', 'ag1', 'ag2']
    elif how == 'g':
        gb_cols = ['Chromosome', 'Strand', 'gene_name',
                   'gene_id']
    gb_cols = list(set(gb_cols)&(set(t_df.columns)))

    cols = gb_cols + ['min_coord', 'max_coord']
    t_df = t_df[cols]
    t_df = t_df.groupby(gb_cols, observed=True).agg(agg_dict).reset_index()
    t_df.rename({'min_coord': 'Start', 'max_coord': 'End'}, axis=1, inplace=True)
    if how == 't':
        t_df['Feature'] = 'transcript'
    elif how == 'g':
        t_df['Feature'] = 'gene'

    return t_df



In [21]:
ifile = 'HG00621_1_novel_gene_rename.gtf'
ref_file = '../../../ref/ref.gtf'
ofile = 'test.gtf'
tool = 'flair'


In [22]:
df = pr.read_gtf(ifile).df

# limit to just transcripts and exons b/c we'll just
# reconstruct the genes
df = df.loc[df.Feature.isin(['transcript', 'exon'])]
gtf_df = df.copy(deep=True)

# flair -- rename antisense genes
if tool == 'flair':
    ref_df = pr.read_gtf(ref_file).df

    # get strand, gene id, and transcript id of flair transcripts
    df = df[['gene_id', 'transcript_id', 'Strand']].drop_duplicates()
    df['gid_stable'] = cerberus.get_stable_gid(df, 'gene_id')

    # get strand and gene id for reference
    ref_df = ref_df[['gene_id', 'Strand']].drop_duplicates()
    ref_df['gid_stable'] = cerberus.get_stable_gid(ref_df, 'gene_id')

    # merge w/ reference to figure out which transcripts are antisense
    df = df.merge(ref_df,
                  how='left',
                  on='gid_stable',
                  suffixes=('', '_ref'))

    # limit to things on the opposite strand as ref
    # and known things
    # and replace gene id for those
    temp = df.loc[(df.Strand!=df.Strand_ref)&\
                  (df.Strand_ref.notnull())]
    tids = temp.transcript_id.unique().tolist()
    inds = gtf_df.loc[gtf_df.transcript_id.isin(tids)].index
    gtf_df.loc[inds, 'gene_id'] = gtf_df.loc[inds, 'gene_id']+'_antisense'

    l1 = len(gtf_df.gene_id.unique())
    l2 = len(gtf_df[['Strand', 'gene_id']].drop_duplicates())

    # make sure we have the same number of gene ids at the end (l2)
    # as we did unique gene id + strand combos at the beginning (l1)
    assert l1 == l2

    # definitionally we are making novel loci by merging 
    # on strands. So make sure we don't have any novel genes
    # w/ antisense designation
    assert len(df.loc[(df.gene_id.str.contains('LOC'))&\
                 (df.gene_id.str.contains('antisense'))].index) == 0

In [23]:
print(len(df.loc[df.gene_id.str.contains('LOC')].index))
print(len(df.loc[(df.gene_id.str.contains('LOC'))&\
                 (df.gene_id.str.contains('antisense'))].index))

3961
0


In [24]:
# make new gene entries
df = gtf_df.copy(deep=True)
l1 = len(df.gene_id.unique().tolist())

# make gene entry
g_df = make_hier_entry(df, how='g')

if tool == 'espresso': source = 'Espresso'
elif tool == 'flair': source = 'FLAIR'
elif tool == 'iq': source = 'IsoQuant'
g_df['Source'] = source
g_df['Frame'] = '.'
g_df['Score'] = '.'
l2 = len(g_df.loc[g_df.Feature=='gene'].index)
assert l1 == l2

# concat them and then sort gtf
df = pd.concat([df, g_df], axis=0)
df = cerberus.sort_gtf(df)

In [25]:
df.head()

,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,gene_id,transcript_id,exon_number
0,chr1,FLAIR,gene,169795042,169804386,.,+,.,ENSG00000000460.17,NaN,NaN
1,chr1,FLAIR,transcript,169795078,169804386,.,+,.,ENSG00000000460.17,98e251b6-26ca-48f1-a06f-0163f0ad2ee0:0-0,NaN
2,chr1,FLAIR,exon,169795078,169795213,.,+,.,ENSG00000000460.17,98e251b6-26ca-48f1-a06f-0163f0ad2ee0:0-0,0
3,chr1,FLAIR,exon,169798918,169798958,.,+,.,ENSG00000000460.17,98e251b6-26ca-48f1-a06f-0163f0ad2ee0:0-0,1
4,chr1,FLAIR,exon,169800882,169800971,.,+,.,ENSG00000000460.17,98e251b6-26ca-48f1-a06f-0163f0ad2ee0:0-0,2


11965
11965


In [26]:
# # iq -- add gene names and transcript names
# if tool == 'iq':

#     # just grab all the gene ids to be the gene names
#     df['gene_name'] = df['gene_id']

#     # for transcripts / other transcript-level features do the
#     # same but restrict to those feats
#     inds = df.loc[df.Feature != 'gene'].index
#     df.loc[inds, 'transcript_name'] = df.loc[inds, 'transcript_id']

# convert + save
df = pr.PyRanges(df)
df.to_gtf(ofile)